In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import zipfile
import glob
import rasterio

### Download the data

In [2]:
# Copernicus scihub login details
user = 'username'
password = 'password'

In [3]:
api = SentinelAPI(
    user=user,
    password=password,
    api_url="https://scihub.copernicus.eu/dhus/",
)

geojson = read_geojson('footprint.geojson')
footprint = geojson_to_wkt(geojson)
date = ('NOW-30DAYS','NOW')
producttype = 'S2MSI2A'

products = api.query(footprint, date=date, producttype=producttype)

In [4]:
product_df = api.to_dataframe(products)\
                .sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])\
                .head(1)

In [5]:
product_df[['cloudcoverpercentage', 'ingestiondate']] # hopefully we have a scotland image with little cloud.

,cloudcoverpercentage,ingestiondate
ff66dbd5-46d4-4d3b-a8e7-07fa9517526e,18.800721,2020-05-27 20:22:46.994


In [6]:
api.download_all(product_df.index)

(OrderedDict([('ff66dbd5-46d4-4d3b-a8e7-07fa9517526e',
               {'id': 'ff66dbd5-46d4-4d3b-a8e7-07fa9517526e',
                'title': 'S2B_MSIL2A_20200527T113319_N0214_R080_T30VVH_20200527T140542',
                'size': 1152621303,
                'md5': 'EF0BEE8E84C532F1680FBB0EC288C698',
                'date': datetime.datetime(2020, 5, 27, 11, 33, 19, 24000),
                'footprint': 'POLYGON((-4.639557 56.833046680181816,-2.8399963 56.843709329150045,-2.8440857 55.85722237128517,-4.597809 55.84694811162493,-4.639557 56.833046680181816))',
                'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('ff66dbd5-46d4-4d3b-a8e7-07fa9517526e')/$value",
                'Online': True,
                'Creation Date': datetime.datetime(2020, 5, 27, 20, 23, 28, 248000),
                'Ingestion Date': datetime.datetime(2020, 5, 27, 20, 22, 46, 994000),
                'path': './S2B_MSIL2A_20200527T113319_N0214_R080_T30VVH_20200527T140542.zip',
              

### Unzip and export 20m resolution bands to GeoTIFF

In [7]:
s2zipfile = product_df.title.tolist()[0] + '.zip'
with zipfile.ZipFile(s2zipfile, 'r') as zip_ref:
    zip_ref.extractall('.')
s2ds = product_df.title.tolist()[0] + '.SAFE'

In [12]:
s2r20m_files = sorted(glob.glob(s2ds + '/GRANULE/**/IMG_DATA/R20m/*_20m.jp2', recursive=True))

In [13]:
bands = {}
ignored_bands = ['AOT', 'TCI', 'WVP', 'SCL'] # we just want the numbered bands
for f in s2r20m_files:
    label = f.split('_')[-2]
    if any([label == ignored_band for ignored_band in ignored_bands]): continue
    bands[label] = rasterio.open(f, driver='JP2OpenJPEG')

In [15]:
#export multiband geotiff image
filename = product_df.title.tolist()[0] + '_20m.tif'
geotiff = rasterio.open(filename,'w',driver='Gtiff',
                         width=bands['B04'].width, height=bands['B04'].height,
                         count=len(bands.keys()),
                         crs=bands['B04'].crs,
                         transform=bands['B04'].transform,
                         dtype=bands['B04'].dtypes[0]
                         )
for i, label in enumerate(bands.keys()):
    geotiff.write(bands[label].read(1), i + 1)
geotiff.close()

In [16]:
list(bands.keys()) # this order

['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B11', 'B12', 'B8A']

We have produced a geotiff from the Sentinel 2 20m resolution data we downloaded. The next step could be to load it into yt and see what we get!